In [29]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.font_manager import FontProperties

# matplotlib.style.use('ggplot')
import matplotlib
# matplotlib.style.use('ggplot')
#matplotlib.style.use('default')
from numpy import *
import numpy as np
%matplotlib inline
%pylab inline
pylab.rcParams['figure.figsize'] = (8, 6)
import subprocess,shlex,sys,select,re,StringIO
def readHive(sql):
    text = ''
    err = ''
    cmd = shlex.split('hive -e '+ '"set mapreduce.job.queuename=zampda_job;set hive.cli.print.header=true;'+sql+'"')
    print >>sys.stderr, cmd
    sys.stderr.flush()
    proc = subprocess.Popen(cmd, bufsize=0, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    while proc.poll() is None:
        ins, outs, errs = select.select([proc.stdout, proc.stderr], [], [])
        if proc.stdout in ins:
            text += proc.stdout.read()
        if proc.stderr in ins:
            c = proc.stderr.read(1)
            if c == '\n':
                print >>sys.stderr, err
                sys.stderr.flush()
                err = ''
            else:
                err += c
    text += proc.stdout.read()
    print >>sys.stderr, proc.stderr.read()
    sys.stderr.flush()
    header, content = text.split('\n',1)
    header = re.sub('\w*\.','', header)
    return pd.read_table(StringIO.StringIO(header+'\n'+content), error_bad_lines = False)

def readD(sql):
    text = ''
    err = ''
    cmd = shlex.split('mysql -h 172.22.16.8 -p 3308 -e "'+sql+'"')
    print >>sys.stderr, cmd
    sys.stderr.flush()
    proc = subprocess.Popen(cmd, bufsize=0, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    while proc.poll() is None:
        ins, outs, errs = select.select([proc.stdout, proc.stderr], [], [])
        if proc.stdout in ins:
            text += proc.stdout.read()
        if proc.stderr in ins:
            c = proc.stderr.read(1)
            if c == '\n':
                print >>sys.stderr, err
                sys.stderr.flush()
                err = ''
            else:
                err += c
    text += proc.stdout.read()
    print >>sys.stderr, proc.stderr.read()
    sys.stderr.flush()
#     header, content = text.split('\n',1)
#     header = re.sub('\w*\.','', header)
    return pd.read_table(StringIO.StringIO(text), error_bad_lines = False)

def f(x):
    x=str(x)
    tmp=x.split("//")
    if len(tmp)<=1:
        return x
    return '.'.join(x.strip().split('//')[1].split('/')[0].split('.')[-2:])

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [15]:
c=readHive("select * from fraud_free.dclkv2 where year=2016 and month=11 and day=8")

['hive', '-e', 'set mapreduce.job.queuename=zampda_job;set hive.cli.print.header=true;select * from fraud_free.dclkv2 where year=2016 and month=11 and day=8']

Logging initialized using configuration in file:/var/bh/lib/apache-hive-1.2.1-bin/conf/hive-log4j.properties
OK
Time taken: 2.96 seconds, Fetched: 673057 row(s)



In [30]:
app=readD("select app_id, sum(clk)/sum(imp) as ctr, sum(win_cpi)/sum(clk) as cpc, sum(imp) from bmms_v4 where __time > \"2016-11-02T00:00:00+0800\" group by app_id")

['mysql', '-h', '172.22.16.8', '-p', '3308', '-e', 'select app_id, sum(clk)/sum(imp) as ctr, sum(win_cpi)/sum(clk) as cpc, sum(imp) from bmms_v4 where __time > 2016-11-02T00:00:00+0800 group by app_id']
user 'changbin'@'TS-BHDEV-01' (using password: NO)



ValueError: No columns to parse from file

In [19]:
app

,"mysql Ver 14 Distrib 24, for Linux (x86_64) using EditLine wrapper"
0,"Copyright (c) 2000, 2011, Oracle and/or its af..."
1,Oracle is a registered trademark of Oracle Cor...
2,affiliates. Other names may be trademarks of t...
3,owners.
4,Usage: mysql [OPTIONS] [database]
5,"-?, --help Display this help and exit."
6,"-I, --help Synonym for -?"
7,--auto-rehash Enable automatic rehashi...
8,'rehash' to get table an...
9,and reconnecting may tak...


In [13]:
c.query('fraud_flag==fraud_flag')['request_id'].count()

57595

In [14]:
c.query('fraud_flag!=fraud_flag')['request_id'].count()

200516

155863

In [8]:
c.query('fraud_flag==fraud_flag')['adslot_id'].value_counts()

mm_26632206_2690592_14738448     2873
9223372032562522990              2777
mm_15890324_2192376_11153358     2732
9223372032561520956              1680
9223372032562004323              1560
9223372032562119501              1543
9223372032562252938              1400
9223372032561345599              1343
9223372032562559157              1307
9223372032562093835              1197
9223372032562020284              1165
9223372032562531153              1159
9223372032561468577              1034
9223372032562530963               957
mm_108636149_8862290_42786782     952
9223372032562519141               942
9223372032562530560               917
9223372032562545443               861
9223372032562435412               856
mm_45054201_4236569_33882611      852
mm_32875384_3406520_40818998      836
7581330927849651016               830
9223372032562416712               826
9223372032562526254               785
9223372032562459023               780
9223372032562406615               760
922337203256

In [5]:
c=readHive("select * from fraud_free.dclkv2 where year=2016 and month=11 and day=7")

TypeError: unsupported operand type(s) for >>: 'builtin_function_or_method' and 'OutStream'

ImportError: No module named 'commands'

In [19]:
c=readHive("select * from archive.rtbv2 where year=2016 and month=11 and day=7 limit 1")

['hive', '-e', 'set mapreduce.job.queuename=zampda_job;set hive.cli.print.header=true;select * from archive.rtbv2 where year=2016 and month=11 and day=7 limit 1']
b'conds, Fetched: 1 row(s)\n'


ValueError: not enough values to unpack (expected 2, got 1)

['hive', '-e', 'set mapreduce.job.queuename=zampda_job;set hive.cli.print.header=true;desc archive.rtbv2']


TypeError: Can't convert 'bytes' object to str implicitly

In [3]:
c=readHive("select * from fraud_free.dclkv2 where year=2016 and month=10 and day=31")

['hive', '-e', 'set mapreduce.job.queuename=zampda_job;set hive.cli.print.header=true;select * from fraud_free.dclkv2 where year=2016 and month=10 and day=31']

Logging initialized using configuration in file:/var/bh/lib/apache-hive-1.2.1-bin/conf/hive-log4j.properties
OK
econds, Fetched: 392191 row(s)



AttributeError: type object '_io.StringIO' has no attribute 'StringIO'

In [5]:
c

NameError: name 'c' is not defined